In [1]:
import tensorflow as tf
print(tf.__version__)
import tf_keras as keras
# from tensorflow import keras
# import tf_keras as keras
# import keras as keras

# from tf.keras.datasets import fashion_mnist
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import to_categorical
from tensorflow.python.keras.layers import Conv1D, Dense, Flatten
# from tensorflow.python.keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np
from pathlib import Path
import os
from sklearn.model_selection import train_test_split

2.19.0



In [3]:
import os
import numpy as np
import pandas as pd
from pathlib import Path



# Parameters
WINDOW_SIZE = 100   # samples per window
STEP_SIZE = 50      # overlap
DATA_DIR = str(Path.cwd().parent.parent / 'Reffit' / 'Processed')   # parent folder with subfolders for each appliance
LABELS = []
X = []
y = []

def process_file(file_path, label_idx):
    df = pd.read_csv(file_path)
    print(f"Processing file: {file_path}, Label index: {label_idx}")
    with open(file_path) as f:
      # Extract appliance name from filename
        appliance_name = f.readline().strip().split(",")[1]
    if appliance_name not in df.columns:
        return

    signal = df[appliance_name].fillna(0).values.astype(np.float32)

    for start in range(0, len(signal) - WINDOW_SIZE, STEP_SIZE):
        window = signal[start:start + WINDOW_SIZE]
        X.append(window)
        y.append(label_idx)

def load_dataset(data_dir):
    global LABELS
    for idx, class_dir in enumerate(sorted(Path(data_dir).iterdir())):
        if not class_dir.is_dir():
            continue
        label_name = class_dir.name
        LABELS.append(label_name)

        for file in class_dir.glob("*.csv"):
            process_file(file, idx)

# Load and preprocess data
load_dataset(DATA_DIR)
X = np.array(X)[..., np.newaxis]  # shape: [samples, window_size, 1]
y = to_categorical(y, num_classes=len(LABELS))


Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Project\Powerverse\NILM_Project\Datasets\Reffit\Processed\Bread-maker\Bread-maker_House_19.csv_11.csv, Label index: 0
Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Project\Powerverse\NILM_Project\Datasets\Reffit\Processed\Bread-maker\Bread-maker_House_19.csv_12.csv, Label index: 0
Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Project\Powerverse\NILM_Project\Datasets\Reffit\Processed\Bread-maker\Bread-maker_House_19.csv_56.csv, Label index: 0
Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Project\Powerverse\NILM_Project\Datasets\Reffit\Processed\Bread-maker\Bread-maker_House_19.csv_62.csv, Label index: 0
Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Project\Powerverse\NILM_Project\Datasets\Reffit\Processed\Computer\Computer_House_1.csv_0.csv, Label index: 1
Processing file: c:\Users\niran\OneDrive\Documents\UL\Edge Computing\Proj

In [4]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
def build_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(32, 5, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(X_train.shape[1:], len(LABELS))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 96, 32)            192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 48, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 64)            6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1472)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                94272     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

In [6]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32
)

AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'